In [2]:
import os
import json

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_text

In [3]:
tf.saved_model.LoadOptions(
    allow_partial_checkpoint=False,
    experimental_io_device='/job:localhost',
    experimental_skip_checkpoint=False
)

tf_model = tf.keras.models.load_model(
    os.getcwd() + '/../../models/USE_model/'
)

In [4]:
train_path = os.getcwd() + '/../../input_data/dataset_train.json'
test_path = os.getcwd() + '/../../input_data/dataset_test.json'

unlabeled_path = os.getcwd() + '/../../input_data/dataset_unlabeled.json'

In [6]:
def read_json_data(filepath):
    
    with open(filepath, 'r') as f:
        data_dict = json.load(f)
        
    return data_dict

def get_values_list_from_dict(data_dict):
    
    data_list = []
    for key, value in data_dict.items():
        data_list.append(value)
        
    return data_list

In [6]:
training_data = get_values_list_from_dict(read_json_data(train_path))
test_data = get_values_list_from_dict(read_json_data(test_path))

In [8]:
train_df = pd.DataFrame(training_data)
test_df = pd.DataFrame(test_data)

In [14]:
train_df.sample(3)

,id,text,label,lang,label_name
41,210705_news_131860,Saab hat das “Artilleriejagd Radar” ARTHUR ART...,2,de,military
35,210705_news_75148,"Auch die US-Regierung setzt nun auf ""vertrauen...",1,de,technology
212,210705_news_492092,Redner der Opposition und der Koalitionsfrakti...,0,de,negative


In [15]:
train_df['doc_vec'] = train_df.apply(lambda x:tf_model(x['text'])['outputs'].numpy()[0].reshape(1, -1), axis=1)
test_df['doc_vec'] = test_df.apply(lambda x:tf_model(x['text'])['outputs'].numpy()[0].reshape(1, -1), axis=1)

In [17]:
train_df.to_pickle(os.getcwd() + '/../dataframes/train_df.pkl')
test_df.to_pickle(os.getcwd() + '/../dataframes/test_df.pkl')

In [7]:
unlabeled_data = get_values_list_from_dict(read_json_data(unlabeled_path))
unlabeled_df = pd.DataFrame(unlabeled_data)
unlabeled_df['doc_vec'] = unlabeled_df.apply(lambda x:tf_model(x['text'])['outputs'].numpy()[0].reshape(1, -1), axis=1)
unlabeled_df.to_pickle(os.getcwd() + '/../dataframes/unlabeled_df.pkl')